# Recreating MTP 2.6 Figure, Population by Generation

## Introduction

This script will recreate the 2.6 figure from 2021 ACS 5-year estimate data

### Process Outline

The process carried out by this workflow can be described as follows:
  - The script will retrieve the Census PEP datasets from the local MORPC repo

  - fdsfsdf

## Setup

### Import Required Packages

In [778]:
import csv
import os
import shutil
import json
from tableschema import Table
import pandas as pd
import xlsxwriter
import matplotlib.pyplot as plt
from pprint import pprint
from frictionless import Detector, describe, validate, Schema
import sys
sys.path.append(os.path.normpath("../morpc-common"))
import morpc

### Parameters

In [779]:
# Assign a color to each of the expected bars in the chart using hex codes.
BAR_COLORS={
        "The Greatest Generation (Born 1945 or earlier)":morpc.CONST_MORPC_COLORS["darkblue"],
        "Baby Boomers (Born 1946 - 1965)":morpc.CONST_MORPC_COLORS["midblue"],
        "Generation X (Born 1966 - 1980)":morpc.CONST_MORPC_COLORS["blue"],
        "Millenials (Born 1981 - 1995)":morpc.CONST_MORPC_COLORS["bluegreen"],
        "Generation Z (Born 1996 - 2009)":morpc.CONST_MORPC_COLORS["darkgreen"],
        "Generation ALPHA (Born 2011 and later)":morpc.CONST_MORPC_COLORS["lightgreen"]
}

CONST_REGIONS = {
    "15-County Region": ["Delaware County, Ohio", "Fairfield County, Ohio", "Fayette", "Franklin", "Hocking", "Knox", "Licking", "Logan", "Madison", "Marion", "Morrow", "Perry", "Pickaway", "Ross", "Union"],
    "10-County Region": ["Delaware County, Ohio", "Fairfield County, Ohio", "Franklin County, Ohio", "Knox County, Ohio", "Licking County, Ohio", "Madison County, Ohio", "Marion County, Ohio", "Morrow County, Ohio", "Pickaway County, Ohio", "Union County, Ohio"],
    "CORPO Region": ["Fairfield", "Knox", "Madison", "Marion", "Morrow", "Pickaway", "Union"]
}

# Define input and output directories
INPUT_DIR = "./input_data"
OUTPUT_DIR = "./output_data"

### Define inputs

#### 2021 ACS 5-year estimates

In [780]:
ACSST5Y2021_TABLE_FILENAME = "ACSST5Y2021.S0101-Data.csv"
ACSST5Y2021_TABLE_PATH = os.path.join(INPUT_DIR, ACSST5Y2021_TABLE_FILENAME)

ACSST5Y2021_TABLE_SCHEMA_FILENAME = "ACSST5Y2021.S0101-Column-Metadata.csv"
ACSST5Y2021_TABLE_SCHEMA_PATH = os.path.join(INPUT_DIR, ACSST5Y2021_TABLE_SCHEMA_FILENAME)

print("Data: {}".format(ACSST5Y2021_TABLE_PATH))
print("Schema: {}".format(ACSST5Y2021_TABLE_SCHEMA_PATH))

Data: ./input_data\ACSST5Y2021.S0101-Data.csv
Schema: ./input_data\ACSST5Y2021.S0101-Column-Metadata.csv


### Define outputs

#### Data used

In [781]:
OUTPUT_TABLE_FILENAME = "output_data.csv"
OUTPUT_TABLE_PATH = os.path.join(OUTPUT_DIR, OUTPUT_TABLE_FILENAME)

#### Compiled Excel sheet

In [782]:
EXCEL_TABLE_FILENAME = "compiled.xlsx"
EXCEL_TABLE_PATH = os.path.join(OUTPUT_DIR, EXCEL_TABLE_FILENAME)
print("Output data path: {}".format(EXCEL_TABLE_PATH))

Output data path: ./output_data\compiled.xlsx


#### Test output

In [783]:
TEST_TABLE_FILENAME = "test_data.csv"
TEST_TABLE_PATH = os.path.join(OUTPUT_DIR, TEST_TABLE_FILENAME)

## Extracting Relevant Data

In [784]:
# Focus on 10-county region
# Read the CSV file into a DataFrame
data = pd.read_csv(ACSST5Y2021_TABLE_PATH, index_col=0)

# Filter out rows where CTYNAME is not in the "10-County Region"
data = data[data['NAME'].isin(CONST_REGIONS["10-County Region"])]

data = data[['S0101_C01_002E', 'S0101_C01_003E','S0101_C01_004E', 'S0101_C01_005E', 'S0101_C01_006E','S0101_C01_007E', 'S0101_C01_008E', 'S0101_C01_009E','S0101_C01_010E', 'S0101_C01_011E', 'S0101_C01_012E','S0101_C01_013E', 'S0101_C01_014E', 'S0101_C01_015E', 'S0101_C01_016E','S0101_C01_017E', 'S0101_C01_018E', 'S0101_C01_019E']]

data = data.rename(columns={'S0101_C01_002E' : 'Under 5 years', 'S0101_C01_003E' : '5 to 9 years','S0101_C01_004E': '10 to 14 years', 'S0101_C01_005E' : '15 to 19 years', 'S0101_C01_006E' : '20 to 24 years','S0101_C01_007E' : '25 to 29 years', 'S0101_C01_008E': '30 to 34 years', 'S0101_C01_009E' : '35 to 39 years','S0101_C01_010E' : '40 to 44 years', 'S0101_C01_011E' : '45 to 49 years', 'S0101_C01_012E' : '50 to 54 years','S0101_C01_013E' : '55 to 59 years', 'S0101_C01_014E' : '60 to 64 years', 'S0101_C01_015E' : '65 to 69 years', 'S0101_C01_016E' : '70 to 74 years','S0101_C01_017E' : '75 to 79 years', 'S0101_C01_018E' : '80 to 84 years', 'S0101_C01_019E' : '85 years and over'})

# Convert all columns in the DataFrame to integer type
data = data.astype(int)

# Sum all columns individually
data = data.sum()

# Convert the Series to a DataFrame
df = data.to_frame().reset_index()

# Now df is a DataFrame, you can rename the columns
df.columns = ['AGEGRP', 'Value']

df.head()

,AGEGRP,Value
0,Under 5 years,141615
1,5 to 9 years,141313
2,10 to 14 years,147091
3,15 to 19 years,145719
4,20 to 24 years,140263


## Combining age ranges by generation

### Gen Alpha (Ages 12 and younger)

In [785]:
test_df= df

# Filter the rows for the 10-14 age group
age_10_14 = test_df[test_df['AGEGRP'] == '10 to 14 years']

# Calculate new values assuming equal proportions for the 10-12 and 13-14 sub-ranges
# Assuming each year within 10-14 has an equal proportion
proportion = 3/5  # 3 years (15-17) out of 5 (15-19)
age_10_12_value = age_10_14['Value'] * proportion
age_13_14_value = age_10_14['Value'] * (2/5)  # Remaining proportion for 2 years (18-19)

# Create new rows for the 15-17 and 18-19 sub-ranges
age_10_12_rows = age_10_14.copy()
age_13_14_rows = age_10_14.copy()

age_10_12_rows['AGEGRP'] = '10 to 12 years'
age_10_12_rows['Value'] = age_10_12_value

age_13_14_rows['AGEGRP'] = '13 to 14 years'
age_13_14_rows['Value'] = age_13_14_value

# Append the new rows to the original DataFrame
new_df = pd.concat([test_df, age_10_12_rows, age_13_14_rows], ignore_index=True)

# Remove the original 15-19 age group rows
new_df = new_df[new_df['AGEGRP'] != '10 to 14 years']
new_df.head()

,AGEGRP,Value
0,Under 5 years,141615.0
1,5 to 9 years,141313.0
3,15 to 19 years,145719.0
4,20 to 24 years,140263.0
5,25 to 29 years,168503.0


In [786]:
# Filter the rows for Under 5 years
age_under_5_value = new_df.loc[new_df['AGEGRP'] == 'Under 5 years', 'Value'].sum()

# Filter the rows for 5 to 9 years
age_5_9_value = new_df.loc[new_df['AGEGRP'] == '5 to 9 years', 'Value'].sum()

# Filter the rows for 10 to 12 year
age_10_12_value = new_df.loc[new_df['AGEGRP'] == '10 to 12 year', 'Value'].sum()

GENALPHA_value = age_under_5_value + age_5_9_value + age_10_12_value 
GENALPHA_row = age_10_12.copy()

GENALPHA_row['AGEGRP'] = 'Generation ALPHA (Born 2011 and later)'
GENALPHA_row['Value'] = GENALPHA_value

final_df = pd.concat([GENALPHA_row,GENZ_row], ignore_index=True)


### Gen Z (Ages 13 to 27)

In [787]:
# Filter the rows for the 25-29 age group
age_25_29 = test_df[test_df['AGEGRP'] == '25 to 29 years']

# Calculate new values assuming equal proportions for the 25-27 and 28-29 sub-ranges
# Assuming each year within 25-29 has an equal proportion
proportion = 3/5  # 3 years (25-27) out of 5 (28-29)
age_25_27_value = age_25_29['Value'] * proportion
age_28_29_value = age_25_29['Value'] * (2/5)  # Remaining proportion for 2 years (18-19)

# Create new rows for the 15-17 and 18-19 sub-ranges
age_25_27_rows = age_25_29.copy()
age_28_29_rows = age_25_29.copy()

age_25_27_rows['AGEGRP'] = '25 to 27 years'
age_25_27_rows['Value'] = age_25_27_value

age_28_29_rows['AGEGRP'] = '28 to 29 years'
age_28_29_rows['Value'] = age_28_29_value

# Append the new rows to the original DataFrame
new_df = pd.concat([new_df, age_25_27_rows, age_28_29_rows], ignore_index=True)

# Remove the original 15-19 age group rows
new_df = new_df[new_df['AGEGRP'] != '25 to 29 years']
print(new_df)

               AGEGRP     Value
0       Under 5 years  141615.0
1        5 to 9 years  141313.0
2      15 to 19 years  145719.0
3      20 to 24 years  140263.0
5      30 to 34 years  167933.0
6      35 to 39 years  155013.0
7      40 to 44 years  143232.0
8      45 to 49 years  139990.0
9      50 to 54 years  137267.0
10     55 to 59 years  136630.0
11     60 to 64 years  128839.0
12     65 to 69 years  104921.0
13     70 to 74 years   77186.0
14     75 to 79 years   47277.0
15     80 to 84 years   31439.0
16  85 years and over   32544.0
17     10 to 12 years   88254.6
18     13 to 14 years   58836.4
19     25 to 27 years  101101.8
20     28 to 29 years   67401.2


In [788]:
# Filter the rows for 13 to 14 years
age_13_14_value = new_df.loc[new_df['AGEGRP'] == '13 to 14 years', 'Value'].sum()

# Filter the rows for 15 to 19 years
age_15_19_value = new_df.loc[new_df['AGEGRP'] == '15 to 19 years', 'Value'].sum()

# Filter the rows for 20 to 24 years
age_20_24_value = new_df.loc[new_df['AGEGRP'] == '20 to 24 years', 'Value'].sum()

# Filter the rows for 25 to 27 years
age_25_27_value = new_df.loc[new_df['AGEGRP'] == '25 to 27 years', 'Value'].sum()

GENZ_value = age_13_14_value + age_15_19_value + age_10_12_value + age_25_27_value

GENZ_row = age_10_12.copy()

GENZ_row['AGEGRP'] = 'Generation Z (Born 1996 - 2009)'
GENZ_row['Value'] = GENZ_value

final_df = pd.concat([GENALPHA_row,GENZ_row], ignore_index=True)

final_df.head()

,AGEGRP,Value
0,Generation ALPHA (Born 2011 and later),282928.0
1,Generation Z (Born 1996 - 2009),305657.2


### Millenials (Ages 28 to 42)

In [789]:
# Filter the rows for the 40-44 age group
age_40_44 = test_df[test_df['AGEGRP'] == '40 to 44 years']

# Calculate new values assuming equal proportions for the 40-42 and 43-44 sub-ranges
# Assuming each year within 40-44 has an equal proportion
proportion = 3/5  # 3 years (40-42) out of 5 (43-44)
age_40_42_value = age_40_44['Value'] * proportion
age_43_44_value = age_40_44['Value'] * (2/5)  # Remaining proportion for 2 years (18-19)

# Create new rows for the 40-42 and 43-44 sub-ranges
age_40_42_rows = age_40_44.copy()
age_43_44_rows = age_40_44.copy()

age_40_42_rows['AGEGRP'] = '40 to 42 years'
age_40_42_rows['Value'] = age_40_42_value

age_43_44_rows['AGEGRP'] = '43 to 44 years'
age_43_44_rows['Value'] = age_43_44_value

# Append the new rows to the original DataFrame
new_df = pd.concat([new_df, age_40_42_rows, age_43_44_rows], ignore_index=True)

# Remove the original 15-19 age group rows
new_df = new_df[new_df['AGEGRP'] != '40 to 44 years']
print(new_df)

               AGEGRP     Value
0       Under 5 years  141615.0
1        5 to 9 years  141313.0
2      15 to 19 years  145719.0
3      20 to 24 years  140263.0
4      30 to 34 years  167933.0
5      35 to 39 years  155013.0
7      45 to 49 years  139990.0
8      50 to 54 years  137267.0
9      55 to 59 years  136630.0
10     60 to 64 years  128839.0
11     65 to 69 years  104921.0
12     70 to 74 years   77186.0
13     75 to 79 years   47277.0
14     80 to 84 years   31439.0
15  85 years and over   32544.0
16     10 to 12 years   88254.6
17     13 to 14 years   58836.4
18     25 to 27 years  101101.8
19     28 to 29 years   67401.2
20     40 to 42 years   85939.2
21     43 to 44 years   57292.8


In [790]:
# Filter the rows for 28 to 29 years
age_28_29_value = new_df.loc[new_df['AGEGRP'] == '28 to 29 years', 'Value'].sum()

# Filter the rows for 30 to 34 years
age_30_34_value = new_df.loc[new_df['AGEGRP'] == '30 to 34 years', 'Value'].sum()

# Filter the rows for 35 to 39 years
age_35_39_value = new_df.loc[new_df['AGEGRP'] == '35 to 39 years', 'Value'].sum()

# Filter the rows for 40 to 42 years
age_40_42_value = new_df.loc[new_df['AGEGRP'] == '40 to 42 years', 'Value'].sum()

MILLENIALS_value = age_28_29_value + age_30_34_value + age_35_39_value + age_40_42_value

MILLENIALS_row = age_40_44.copy()

MILLENIALS_row['AGEGRP'] = 'Millenials (Born 1981 - 1995)'
MILLENIALS_row['Value'] = MILLENIALS_value

final_df = pd.concat([GENALPHA_row, GENZ_row, MILLENIALS_row], ignore_index=True)

final_df.head()

,AGEGRP,Value
0,Generation ALPHA (Born 2011 and later),282928.0
1,Generation Z (Born 1996 - 2009),305657.2
2,Millenials (Born 1981 - 1995),476286.4


### Generation X (Ages 43 to 57)

In [791]:
# Filter the rows for the 55-59 age group
age_55_59 = test_df[test_df['AGEGRP'] == '55 to 59 years']

# Calculate new values assuming equal proportions for the 55-57 and 58-59 sub-ranges
# Assuming each year within 55-59 has an equal proportion
proportion = 3/5  # 3 years (55-57) out of 5 (58-59)
age_55_57_value = age_55_59['Value'] * proportion
age_58_59_value = age_55_59['Value'] * (2/5)  # Remaining proportion for 2 years (18-19)

# Create new rows for the 40-42 and 43-44 sub-ranges
age_55_57_rows = age_55_59.copy()
age_58_59_rows = age_55_59.copy()

age_55_57_rows['AGEGRP'] = '55 to 57 years'
age_55_57_rows['Value'] = age_55_57_value

age_58_59_rows['AGEGRP'] = '58 to 59 years'
age_58_59_rows['Value'] = age_58_59_value

# Append the new rows to the original DataFrame
new_df = pd.concat([test_df,age_43_44_rows, age_55_57_rows, age_58_59_rows], ignore_index=True)

# Remove the original 15-19 age group rows
new_df = new_df[new_df['AGEGRP'] != '55 to 59 years']
print(new_df)

               AGEGRP     Value
0       Under 5 years  141615.0
1        5 to 9 years  141313.0
2      10 to 14 years  147091.0
3      15 to 19 years  145719.0
4      20 to 24 years  140263.0
5      25 to 29 years  168503.0
6      30 to 34 years  167933.0
7      35 to 39 years  155013.0
8      40 to 44 years  143232.0
9      45 to 49 years  139990.0
10     50 to 54 years  137267.0
12     60 to 64 years  128839.0
13     65 to 69 years  104921.0
14     70 to 74 years   77186.0
15     75 to 79 years   47277.0
16     80 to 84 years   31439.0
17  85 years and over   32544.0
18     43 to 44 years   57292.8
19     55 to 57 years   81978.0
20     58 to 59 years   54652.0


In [792]:
# Filter the rows for 43 to 44 years
age_43_44_value = new_df.loc[new_df['AGEGRP'] == '43 to 44 years', 'Value'].sum()

# Filter the rows for 45 to 49 years
age_45_49_value = new_df.loc[new_df['AGEGRP'] == '45 to 49 years', 'Value'].sum()

# Filter the rows for 50 to 54 years
age_50_54_value = new_df.loc[new_df['AGEGRP'] == '50 to 54 years', 'Value'].sum()

# Filter the rows for 55 to 57 yearss
age_55_57_value = new_df.loc[new_df['AGEGRP'] == '55 to 57 years', 'Value'].sum()

GENX_value = age_43_44_value + age_45_49_value + age_50_54_value + age_55_57_value

GENX_row = age_40_44.copy()

GENX_row['AGEGRP'] = 'Generation X (Born 1966 - 1980)'
GENX_row['Value'] = GENX_value

final_df = pd.concat([GENALPHA_row, GENZ_row, MILLENIALS_row, GENX_row], ignore_index=True)

final_df.head()

,AGEGRP,Value
0,Generation ALPHA (Born 2011 and later),282928.0
1,Generation Z (Born 1996 - 2009),305657.2
2,Millenials (Born 1981 - 1995),476286.4
3,Generation X (Born 1966 - 1980),416527.8


### Baby Boomer (58 to 77)

In [793]:
# Filter the rows for the 75 to 79 years age group
age_75_79 = test_df[test_df['AGEGRP'] == '75 to 79 years']

# Calculate new values assuming equal proportions for the 75-77 and 78-79 sub-ranges
# Assuming each year within 75 to 79 has an equal proportion
proportion = 3/5  # 3 years (75-77) out of 5 (78-79)
age_75_77_value = age_75_79['Value'] * proportion
age_78_79_value = age_75_79['Value'] * (2/5)  # Remaining proportion for 2 years (18-19)

# Create new rows for the 40-42 and 43-44 sub-ranges
age_75_77_rows = age_75_79.copy()
age_78_79_rows = age_75_79.copy()

age_75_77_rows['AGEGRP'] = '75 to 77 years'
age_75_77_rows['Value'] = age_75_77_value

age_78_79_rows['AGEGRP'] = '78 to 79 years'
age_78_79_rows['Value'] = age_78_79_value

# Append the new rows to the original DataFrame
new_df = pd.concat([test_df,age_58_59_rows, age_75_77_rows, age_78_79_rows], ignore_index=True)

# Remove the original 15-19 age group rows
new_df = new_df[new_df['AGEGRP'] != '55 to 59 years']
print(new_df)

               AGEGRP     Value
0       Under 5 years  141615.0
1        5 to 9 years  141313.0
2      10 to 14 years  147091.0
3      15 to 19 years  145719.0
4      20 to 24 years  140263.0
5      25 to 29 years  168503.0
6      30 to 34 years  167933.0
7      35 to 39 years  155013.0
8      40 to 44 years  143232.0
9      45 to 49 years  139990.0
10     50 to 54 years  137267.0
12     60 to 64 years  128839.0
13     65 to 69 years  104921.0
14     70 to 74 years   77186.0
15     75 to 79 years   47277.0
16     80 to 84 years   31439.0
17  85 years and over   32544.0
18     58 to 59 years   54652.0
19     75 to 77 years   28366.2
20     78 to 79 years   18910.8


In [794]:
# Filter the rows for 58 to 59 years
age_58_59_value = new_df.loc[new_df['AGEGRP'] == '58 to 59 years', 'Value'].sum()

# Filter the rows for 60 to 64 years
age_60_64_value = new_df.loc[new_df['AGEGRP'] == '60 to 64 years', 'Value'].sum()

# Filter the rows for 65 to 69 years
age_65_69_value = new_df.loc[new_df['AGEGRP'] == '65 to 69 years', 'Value'].sum()

# Filter the rows for 70 to 74 years
age_70_74_value = new_df.loc[new_df['AGEGRP'] == '70 to 74 years', 'Value'].sum()

# Filter the rows for 75 to 77 years 
age_75_77_value = new_df.loc[new_df['AGEGRP'] == '75 to 77 years ', 'Value'].sum()

BABYBOOMERS_value = age_58_59_value + age_60_64_value + age_65_69_value + age_70_74_value + age_75_77_value

BABYBOOMERS_row = age_75_79.copy()

BABYBOOMERS_row['AGEGRP'] = 'Baby Boomers (Born 1946 - 1965)'
BABYBOOMERS_row['Value'] = BABYBOOMERS_value

final_df = pd.concat([GENALPHA_row, GENZ_row, MILLENIALS_row, GENX_row, BABYBOOMERS_row], ignore_index=True)

final_df.head()

,AGEGRP,Value
0,Generation ALPHA (Born 2011 and later),282928.0
1,Generation Z (Born 1996 - 2009),305657.2
2,Millenials (Born 1981 - 1995),476286.4
3,Generation X (Born 1966 - 1980),416527.8
4,Baby Boomers (Born 1946 - 1965),365598.0


### The Greatest Generation (78+)

In [795]:
# Filter the rows for the 75 to 79 years age group
age_temp = test_df[test_df['AGEGRP'] == '85 years and over']

# Filter the rows for 78 to 79 years
age_78_79_value = new_df.loc[new_df['AGEGRP'] == '78 to 79 years', 'Value'].sum()

# Filter the rows for 80 to 84 years
age_80_84_value = new_df.loc[new_df['AGEGRP'] == '80 to 84 years', 'Value'].sum()

# Filter the rows for 85 years and over
age_over_84_value = new_df.loc[new_df['AGEGRP'] == '85 years and over', 'Value'].sum()

GREATESTGENERTAION_value = age_78_79_value + age_80_84_value + age_over_84_value 

GREATESTGENERTAION_row = age_temp.copy()

GREATESTGENERTAION_row['AGEGRP'] = 'The Greatest Generation (Born 1945 or earlier)'
GREATESTGENERTAION_row['Value'] = GREATESTGENERTAION_value

final_df = pd.concat([GREATESTGENERTAION_row, BABYBOOMERS_row, GENX_row, MILLENIALS_row, GENZ_row, GENALPHA_row], ignore_index=True)


# Save the data to the output directory
final_df.to_csv(OUTPUT_TABLE_PATH)

## Creating Excel table and chart

In [796]:
final_df['Proportion'] = final_df['Value'] / final_df['Value'].sum() * 100

final_df['Value'] = final_df['Value'] / 1000

# Create a new Excel file and add a worksheet
workbook = xlsxwriter.Workbook(EXCEL_TABLE_PATH)
worksheet = workbook.add_worksheet()

# Add the column headers
worksheet.write('A1', 'AGEGRP')
worksheet.write('B1', 'Value')
worksheet.write('C1', 'Proportion')

# Write the data to the worksheet
for i, row in enumerate(final_df.itertuples(), 1):
    worksheet.write(i, 0, row.AGEGRP)
    worksheet.write(i, 1, row.Value)
    worksheet.write_number(i, 2, row.Proportion, workbook.add_format({'num_format': '0.00"%"'}))

# Create a chart object of type column
chart = workbook.add_chart({'type': 'column'})

# Configure the series for the first 6 rows only
chart.add_series({
    'name':       'Proportion',
    'categories': '=Sheet1!$A$2:$A$7',
    'values':     '=Sheet1!$B$2:$B$7',
   # 'data_labels': {'value': True, 'num_format': '0.00%', 'position': 'outside_end'},
    'points': [
        {'fill': {'color': morpc.CONST_MORPC_COLORS["darkblue"]}},
        {'fill': {'color': morpc.CONST_MORPC_COLORS["midblue"]}},
        {'fill': {'color': morpc.CONST_MORPC_COLORS["blue"]}},
        {'fill': {'color': morpc.CONST_MORPC_COLORS["bluegreen"]}},
        {'fill': {'color': morpc.CONST_MORPC_COLORS["darkgreen"]}},
        {'fill': {'color': morpc.CONST_MORPC_COLORS["lightgreen"]}},
    ],
    'gap':        50, 
})


# Chart dimensions
chart_width = 1050  # Width, in pixels
chart_height = 750  # Height, in pixels

chart.set_title({
    'name': 'FIGURE 2.6 POPULATION BY GENERATION',
    'name_font': {'name': 'Arial', 'size': 14}
})
# Remove the x-axis labels
chart.set_x_axis({
    'name': '',
    'name_font': {'name': 'Arial', 'size': 10},  # For the axis title
    'num_font': {'name': 'Arial', 'size': 1, 'color': 'white'},  # For the axis labels, though they are minimized
    'major_tick_mark': 'none',
    'minor_tick_mark': 'none',
})

# Set the chart gridlines to faint gray
chart.set_y_axis({
    'name': 'Thousands',
    'name_font': {'name': 'Arial', 'size': 10},
    'num_font': {'name': 'Arial', 'size': 10},  # For the axis labels
    'major_gridlines': {
        'visible': True,
        'line': {'color': '#D0D0D0', 'width': 1}
    },
    'major_tick_mark': 'none',
})

chart.set_legend({
    'font': {'name': 'Arial', 'size': 10},
    'position': 'bottom'
})

# Insert the chart into the worksheet
worksheet.insert_chart('D2', chart, {'x_scale': 1.8, 'y_scale': 1.5})

# Close the workbook to save it
workbook.close()

In [797]:
print(final_df)

                                           AGEGRP     Value  Proportion
0  The Greatest Generation (Born 1945 or earlier)   82.8938    4.295258
1                 Baby Boomers (Born 1946 - 1965)  365.5980   18.943969
2                 Generation X (Born 1966 - 1980)  416.5278   21.582968
3                   Millenials (Born 1981 - 1995)  476.2864   24.679443
4                 Generation Z (Born 1996 - 2009)  305.6572   15.838053
5          Generation ALPHA (Born 2011 and later)  282.9280   14.660308
